In [ ]:
import urllib.request as urllib
urllib.urlretrieve ("https://kannu.csc.fi/index.php/s/2FKIuCYWfzdc2hn/download", "HP.txt")
urllib.urlretrieve ("https://kannu.csc.fi/index.php/s/oU72rsNmnDqJn0d/download", "GoT.txt")

In [ ]:
from pyspark import SparkContext
sc = SparkContext()

## Word Count

#### Load the RDD from the text file

In [ ]:
rdd_input = sc.textFile('HP.txt')
# Split the text into words and flatten the results. Why?
words = rdd_input.flatMap(lambda line: line.split())
words.collect()

#### Map the words in such a way, so that they can be counted later by aggregation

In [ ]:
words_map = words.map(lambda x: (x,1))

#### Apply reduceByKey to complete the counting

In [ ]:
words_count = words_map.reduceByKey(lambda a,b: a+b)
words_count.collect()

#### Sort them in the order of counts

In [ ]:
words_count_sorted_freq = words_count.sortBy(lambda x: x[1])
# words_count_sorted_freq = words_count.sortBy(lambda x: -x[1]) # Descending
words_count_sorted_freq.collect()

## Inverted Index

In [ ]:
# name of the file is GoT.txt
# data cleaning and preparation: get rid of the full stop at the end of sentences, convert everything to lowercase

rdd_got = sc.textFile('GoT.txt')
rdd_got = rdd_got.map(lambda line: line.replace('.', '')).map(lambda line: line.lower())
rdd_got = rdd_got.zipWithIndex()  # Generate the positional index
rdd_parsed = rdd_got.map(lambda x: (x[0].split(), x[1]))  # Split the words
print(rdd_parsed.collect())
# Add the index of the document to each of the words

def add_index_to_words(item):
    result = []
    for word in item[0]:
        result.append((word, item[1]))
    return result

# rdd_parsed_indexed = rdd_parsed.flatMap(lambda x: map(lambda y: (y, x[1]), x[0]))
rdd_parsed_indexed = rdd_parsed.flatMap(add_index_to_words)
rdd_parsed_indexed.collect()

# use the same concept from word count to provide each key a default initial count of 1
rdd_doc_countmap = rdd_parsed_indexed.map(lambda item: (item, 1))
rdd_doc_wordcount = rdd_doc_countmap.reduceByKey(lambda a,b : a+b)

rdd_wordcount_doc = rdd_doc_wordcount.map(lambda item: (item[0][0], [item[0][1], item[1]]))
rdd_grouped = rdd_wordcount_doc.groupByKey()
rdd_grouped = rdd_grouped.mapValues(list)
rdd_sorted = rdd_grouped.sortByKey()
rdd_sorted.collect()